# 4. Diseño de filtros digitales con computadora

In [ ]:
#NOTE importar bibliotecas
import numpy             as np
import matplotlib.pyplot as plt

from scipy        import signal
from scipy.signal import butter

In [ ]:
# Definicion de funcion para graficar respuesta en frecuencia del filtro
def freqz_plot(b,a,sr):
    w, h = signal.freqz(b,a)
    h[h==0] = 1E-5
    
    H    = 20*np.log10( np.abs(h) )
    W    = np.angle  (h)
    W    = np.unwrap (W)
    W    = np.degrees(W)
    w    = np.linspace(0,sr/2,H.shape[0] )
    
    return w, W, H

In [ ]:
# Definicion de funcion para graficar respuesta en frecuencia del filtro
def freqs_plot(b,a):
    w, h = signal.freqs(b,a,worN=np.logspace(-1, 2, 1000) )
    H    = 20*np.log10( np.abs(h) )
    W    = np.angle  (h)
    W    = np.unwrap (W)
    W    = np.degrees(W)
    w    = np.logspace(-1, 2, 1000)#np.linspace(0,sr/2,H.shape[0] )
    
    return w, W, H

### Ejemplo

Diseñe un filtro digital orden 2 tipo Butterworth para una frecuencia de corte 10 Hz y una frecuecia de muestreo de 50 Hz.

In [ ]:
# Definicion de parametros
n  = t
fc = 32.
fs = 120.
wc = 2*np.pi*fc

**Diseño** de un filtro en el domino analógico 

In [ ]:
k   = 2.*fs
wca = k*np.tan(np.pi*fc/fs)

b   = [   wca**2,
        2*wca**2,
          wca**2  ]
a   = [  k**2 + np.sqrt(2)*wca*k+ wca**2,
        2*( wca**2 - k**2),
         k**2 - np.sqrt(2)*wca*k + wca**2 ]

print "[ %s\n  %s  ]"%(b,a)
print k
print wca

In [ ]:
ad  = np.array(a)
bd  = np.array(b)

amax = ad[0]
ad  /= amax
bd  /= amax

print "[ %s\n  %s  ]"%(bd,ad)

In [ ]:
w, W, H, = freqz_plot(bd,ad,fs)

plt.plot(w,H,'b',linewidth=3)
plt.grid(True)
plt.show()

**Diseño** completo en computadora

In [ ]:
b,a = butter(n, wc, analog=True, output='ba')
w_s, W_s, H_s = freqs_plot(b,a)

plt.plot(w_s,H_s,'b',linewidth=3)

plt.grid(True)
plt.show()

In [ ]:
bd, ad = signal.bilinear(b,a,fs)
print '[%s]\n[%s]'%(bd,ad)

In [ ]:
w_z, W_z, H_z = freqz_plot(bd,ad,fs)

In [ ]:
plt.plot(w_s,400*H_s,'b',linewidth=3)
plt.plot(w_z,H_z,'r',linewidth=3)

plt.grid(True)
plt.show()

In [ ]:
path  = '../../data/'
fname = 'ecg.npz'

fdata = np.load( path+fname )
data  = fdata['arr_0']
fs    = 120.
t     = np.linspace( 0,data.shape[0]/fs,data.shape[0] )

tshow = 1000
plt.figure(u'Señal',figsize=(12,5))
plt.grid(True)
plt.plot( t[:tshow],-data[:tshow],linewidth=2 )

plt.figure(u'PSD',figsize=(12,5))
plt.psd( data[:tshow],Fs=fs,NFFT=4*int(fs),linewidth=3 )
plt.show()

In [ ]:
n   = 2
fc  = 1.2#*(2./fs)
b,a = butter(n, fc, analog=True,btype='high', output='ba')

print('a: %s\nb: %s'%(a,b))
b,a = signal.bilinear(b,a,fs)

w, W, H, = freqz_plot(b,a,fs)

plt.figure(u'Respuesta en frecuencia',figsize=(12,5))
plt.plot(w,H,'b',linewidth=3)
#plt.plot(w[300:],H[300:],'b',linewidth=3)
plt.grid(True)
plt.show()


In [ ]:
data_f = signal.lfilter(b,a,data)

tshow = 500
plt.figure(u'Señal',figsize=(12,5))
plt.grid(True)
plt.plot( t[:tshow], -data  [:tshow],'b',linewidth=2 )
plt.plot( t[:tshow],-data_f[:tshow],'r',linewidth=2 )

plt.figure(u'PSD',figsize=(12,5))
plt.psd( data  [:tshow],Fs=fs,NFFT=4*int(fs),linewidth=3,color='b' )
plt.psd( data_f[:tshow],Fs=fs,NFFT=4*int(fs),linewidth=3,color='r' )

plt.show()

In [ ]:
"""
 @note: ¿qué pasa para n muy grande?
"""
n   = 3
fc  = 2*np.array( [1.2, 8.] )/fs
b,a = butter(n, fc, btype='band', output='ba')

w, W, H, = freqz_plot(b,a,fs)

plt.figure(u'Respuesta en frecuencia dB',figsize=(12,5))
plt.plot(w,H,'b',linewidth=3)
plt.grid(True)
plt.figure(u'Respuesta en frecuencia Hz',figsize=(12,5))
plt.plot(w,W,'g',linewidth=3)
plt.grid(True)
plt.show()



In [ ]:
print(b)
print(a)

In [ ]:
data_f = signal.lfilter(b,a,data)

tshow = 1000
plt.figure(u'Señal',figsize=(12,5))
plt.grid(True)
plt.plot( t[:tshow], -data  [:tshow],'b',linewidth=2 )
plt.plot( t[:tshow], data_f[:tshow],'r',linewidth=2 )

plt.figure(u'PSD',figsize=(12,5))
plt.psd( data  [:tshow],Fs=fs,NFFT=4*int(fs),linewidth=3,color='b' )
plt.psd( data_f[:tshow],Fs=fs,NFFT=4*int(fs),linewidth=3,color='r' )

plt.show()

## Fase cero

In [ ]:
tshow   = 600
data_ff = signal.lfilter(b,a,data_f[::-1])[::-1]

plt.figure(u'Señal',figsize=(15,5))
plt.grid(True)
plt.plot( t[:tshow],-data_ff[:tshow],'r',linewidth=2 )
plt.plot( t[:tshow],-data_f [:tshow],'b',linewidth=2 )
plt.plot( t[:tshow],-data   [:tshow],'g',linewidth=2 )

plt.figure(u'PSD',figsize=(12,5))
plt.psd( data_f [:tshow],Fs=fs,NFFT=4*int(fs),linewidth=3,color='b' )
plt.psd( data_ff[:tshow],Fs=fs,NFFT=4*int(fs),linewidth=3,color='r' )
plt.psd( data   [:tshow],Fs=fs,NFFT=4*int(fs),linewidth=3,color='g' )

plt.show()